<a href="https://colab.research.google.com/github/gabriellaaileen/tensorflow-certification/blob/main/Question5_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

No. 5 Time Series

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # Deep learning library
import matplotlib.pyplot as plt # Visualization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Cryptocurrency2020.csv')
df

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1.618880e+12,4/20/2021 0:02,BTCUSD,55717.47,55723.00,55541.69,55541.69,2.406734
1,1.618880e+12,4/20/2021 0:01,BTCUSD,55768.94,55849.82,55711.74,55717.47,0.573458
2,1.618880e+12,4/20/2021 0:00,BTCUSD,55691.79,55793.15,55691.79,55768.94,3.309904
3,1.618880e+12,4/19/2021 23:59,BTCUSD,55777.86,55777.86,55677.92,55691.79,0.684934
4,1.618880e+12,4/19/2021 23:58,BTCUSD,55803.50,55823.88,55773.08,55777.86,0.477225
...,...,...,...,...,...,...,...,...
658060,1.577840e+12,1/1/2020 0:04,BTCUSD,7163.60,7163.60,7163.00,7163.00,1.751844
658061,1.577840e+12,1/1/2020 0:03,BTCUSD,7163.00,7163.60,7163.00,7163.60,0.002706
658062,1.577840e+12,1/1/2020 0:02,BTCUSD,7163.30,7164.22,7163.00,7163.00,0.063904
658063,1.577840e+12,1/1/2020 0:01,BTCUSD,7163.30,7164.23,7163.30,7163.30,0.002646


In [4]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    if shuffle_buffer > 0:
        ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

train, val = train_test_split(df.iloc[:, 3:], test_size=0.2, shuffle=False)
scaler = MinMaxScaler().fit(train)
train = scaler.transform(train)[:, 0]
val = scaler.transform(val)[:, 0]
train = windowed_dataset(train, window_size=64, batch_size=128, shuffle_buffer=1000)
val = windowed_dataset(val, window_size=64, batch_size=128, shuffle_buffer=1000)

In [5]:
train

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>

In [6]:
val

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>

In [12]:
#Buatlah model dengan input layer disarankan menggunakan LSTM LSTM dan output layer 1 
model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[64]),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(1)
])

In [13]:
# Initialize the optimizer
optimizer = tf.keras.optimizers.Adam()

#gunakan loss MAE, optimizer boleh bebas memilih
model.compile(
    loss=tf.keras.losses.MeanAbsoluteError(), optimizer=optimizer
)

In [9]:
callbacks=[
        tf.keras.callbacks.EarlyStopping(
            patience=10, 
            restore_best_weights=True
        ), 
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=5
        )
    ]

In [14]:
#silakan menentukan banyaknya epoch sendiri
history = model.fit(
    train, steps_per_epoch=64, epochs=50, batch_size=32, validation_data=val, verbose=1, callbacks=[callbacks]
)

Epoch 1/50
64/64 [==============================] - 56s 727ms/step - loss: 0.0670 - val_loss: 0.2105 - lr: 0.0010
Epoch 2/50
64/64 [==============================] - 28s 442ms/step - loss: 0.0183 - val_loss: 0.1825 - lr: 0.0010
Epoch 3/50
64/64 [==============================] - 28s 448ms/step - loss: 0.0131 - val_loss: 0.1949 - lr: 0.0010
Epoch 4/50
64/64 [==============================] - 30s 469ms/step - loss: 0.0136 - val_loss: 0.1699 - lr: 0.0010
Epoch 5/50
64/64 [==============================] - 28s 442ms/step - loss: 0.0160 - val_loss: 0.1088 - lr: 0.0010
Epoch 6/50
64/64 [==============================] - 28s 443ms/step - loss: 0.0125 - val_loss: 0.0273 - lr: 0.0010
Epoch 7/50
64/64 [==============================] - 30s 476ms/step - loss: 0.0153 - val_loss: 0.0847 - lr: 0.0010
Epoch 8/50
64/64 [==============================] - 28s 446ms/step - loss: 0.0151 - val_loss: 0.0958 - lr: 0.0010
Epoch 9/50
64/64 [==============================] - 28s 444ms/step - loss: 0.0119 - val_

In [15]:
%cd /content/drive/MyDrive/Model

/content/drive/MyDrive/Model


In [16]:
#save model menjadi .h5
model.save('model5.h5')